<a href="https://colab.research.google.com/github/CarlosMejia07/Proyecto-Kaggle-UdeA-20252/blob/main/04%20-%20Second_attempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**En este modelo se usa el preprocesado de la segunda entrega sin el one-hot encoding y la predicción con Catboost. Puntaje = 0.38973**

**Importe, descompresión e inspección de los datos**

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia
!unzip udea*.zip > /dev/null
!wc *.csv

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.35GB/s]
   296787    296787   4716673 submission_example.csv
   296787   4565553  59185238 test.csv
   692501  10666231 143732437 train.csv
  1286075  15528571 207634348 total


**Importe de las librerias a usar y lectura del archivo train**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install catboost

z = pd.read_csv("train.csv")
print ("Estructura de los datos cargados", z.shape)
z_clean = z.copy()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.4 MB/s eta 0:00:00
Estructura de los datos cargados (692500, 21)


**Preprocesado de train**

In [ ]:
#Se dejan de lado las columnas que no aporten a la predicción, preliminarmente
z_clean = z_clean.drop(columns=['ID', 'PERIODO_ACADEMICO'])

#Detección y reemplazo de valores nulos
# Separar valores numéricos y no numéricos
numeric_cols = z_clean.select_dtypes(include=np.number).columns
non_numeric_cols = z_clean.select_dtypes(exclude=np.number).columns

# Reemplazar valores nulos de columnas numéricas con la media
z_clean[numeric_cols] = z_clean[numeric_cols].fillna(z_clean[numeric_cols].median())

# Reemplazar valores nulos de columnas no numéricas por la moda
for col in non_numeric_cols:
    z_clean[col] = z_clean[col].fillna(z_clean[col].mode()[0])

z_clean.isna().sum()

#Codificación de las variables con valores nominales y ordinales
def replace_binary(col):
    return col.replace({
        'Si': 1, 'Sí': 1, 'S': 1, 'Y': 1, '1': 1, 'si': 1, 'sí': 1,
        'No': 0, 'N': 0, '0': 0, 'no': 0
    })

binary_cols = ['F_TIENEINTERNET', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL',
               'E_PRIVADO_LIBERTAD', 'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR',
               'F_TIENEINTERNET.1']

for col in binary_cols:
    z_clean[col] = replace_binary(z_clean[col])

z_clean[binary_cols] = z_clean[binary_cols].astype(int)

# Reemplazar valores de variables ordinales
z_clean['F_EDUCACIONPADRE'] = z_clean['F_EDUCACIONPADRE'].replace(['No sabe', 'No Aplica'], 'Desconocido')
z_clean['F_EDUCACIONMADRE'] = z_clean['F_EDUCACIONMADRE'].replace(['No sabe', 'No Aplica'], 'Desconocido')

orden_educacion = [
    'Desconocido',
    'Ninguno',
    'Primaria incompleta',
    'Primaria completa',
    'Secundaria (Bachillerato) incompleta',
    'Secundaria (Bachillerato) completa',
    'Técnica o tecnológica incompleta',
    'Técnica o tecnológica completa',
    'Educación profesional incompleta',
    'Educación profesional completa',
    'Postgrado'
]
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder(categories=[orden_educacion])
z_clean['F_EDUCACIONPADRE'] = encoder.fit_transform(z_clean[['F_EDUCACIONPADRE']])
z_clean['F_EDUCACIONMADRE'] = encoder.fit_transform(z_clean[['F_EDUCACIONMADRE']])

orden_matrícula = [
    'No pagó matrícula',
    'Menos de 500 mil',
    'Entre 500 mil y menos de 1 millón',
    'Entre 1 millón y menos de 2.5 millones',
    'Entre 2.5 millones y menos de 4 millones',
    'Entre 4 millones y menos de 5.5 millones',
    'Entre 5.5 millones y menos de 7 millones',
    'Más de 7 millones'
]
encoder = OrdinalEncoder(categories=[orden_matrícula])
z_clean['E_VALORMATRICULAUNIVERSIDAD'] = encoder.fit_transform(z_clean[['E_VALORMATRICULAUNIVERSIDAD']])

orden_estrato = [
    'Sin Estrato',
    'Estrato 1',
    'Estrato 2',
    'Estrato 3',
    'Estrato 4',
    'Estrato 5',
    'Estrato 6'
]
encoder = OrdinalEncoder(categories=[orden_estrato])
z_clean['F_ESTRATOVIVIENDA'] = encoder.fit_transform(z_clean[['F_ESTRATOVIVIENDA']])

orden_trabajo = [
    '0',
    'Menos de 10 horas',
    'Entre 11 y 20 horas',
    'Entre 21 y 30 horas',
    'Más de 30 horas'
]
encoder = OrdinalEncoder(categories=[orden_trabajo])
z_clean['E_HORASSEMANATRABAJA'] = encoder.fit_transform(z_clean[['E_HORASSEMANATRABAJA']])

#Estandarizado de las variables ordinales, necesario o no según el o los modelos que se usarán
# Con media = 0, desviación estándar = 1
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
cols_ordinales = ['E_HORASSEMANATRABAJA', 'F_ESTRATOVIVIENDA',
                  'E_VALORMATRICULAUNIVERSIDAD', 'F_EDUCACIONPADRE', 'F_EDUCACIONMADRE']

z_clean[cols_ordinales] = scaler.fit_transform(z_clean[cols_ordinales])

# Variables numéricas
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
cols_numericas = ['INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']

z_clean[cols_numericas] = scaler.fit_transform(z_clean[cols_numericas])

#Variable objetivo
# Definimos el orden lógico
orden = ['bajo', 'medio-bajo', 'medio-alto', 'alto']

z_clean['RENDIMIENTO_GLOBAL'] = pd.Categorical(
    z_clean['RENDIMIENTO_GLOBAL'],
    categories=orden,
    ordered=True
).codes

# Visualización de los datos
z_clean.head(10)

/tmp/ipython-input-2513527682.py:20: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return col.replace({


,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,ENFERMERIA,BOGOTÁ,1.383918,-0.971956,0.456215,1,0.356101,1,1,0,0,1,1,1.795668,2,0.437002,-0.556223,0.813978,0.060296
1,DERECHO,ATLANTICO,0.270116,-1.628413,0.456215,0,0.714988,1,0,0,0,1,0,0.236453,0,0.346934,-0.481341,0.508180,0.016142
2,MERCADEO Y PUBLICIDAD,BOGOTÁ,0.270116,0.997418,0.456215,1,-0.002786,1,0,0,0,0,1,-0.153350,0,0.232301,-0.492038,0.729034,0.016142
3,ADMINISTRACION DE EMPRESAS,SANTANDER,0.827017,-1.628413,1.363766,1,-1.797221,1,0,0,0,1,1,-0.153350,3,1.771650,-0.941332,-0.171371,-1.072993
4,PSICOLOGIA,ANTIOQUIA,0.270116,0.340960,0.456215,1,-0.720560,1,1,0,0,1,1,-0.932957,1,0.387874,-0.299484,0.389259,0.457683
5,MEDICINA VETERINARIA,ANTIOQUIA,1.940819,-0.971956,2.271316,1,1.432762,1,1,0,0,1,1,-0.153350,2,2.328436,-1.262255,-0.239326,-1.293764
6,INGENIERIA MECANICA,HUILA,0.270116,0.340960,-0.451335,1,1.073875,1,1,0,1,1,1,0.626257,3,-0.218040,0.417246,0.083461,0.663736
7,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,-0.286785,-0.315498,-0.451335,1,-1.079447,1,0,0,1,0,1,-0.543154,1,-0.250792,0.599102,0.406247,0.398811
8,INGENIERIA INDUSTRIAL,ATLANTICO,1.383918,-0.971956,-1.358885,1,-0.002786,1,1,0,1,1,1,1.016060,1,-0.995902,1.572571,-0.850922,0.958097
9,ADMINISTRACION DE EMPRESAS,ANTIOQUIA,0.270116,0.997418,2.271316,1,1.791650,1,1,0,1,1,1,1.795668,3,2.181051,-1.476204,-0.086427,-1.411508


In [ ]:
object_cols = X.select_dtypes(include='object').columns
print(object_cols)

Index([], dtype='object')


***Modelado***

In [ ]:
#Variable predictoria y objetivo
X = z_clean.drop(columns=['RENDIMIENTO_GLOBAL'])
y = z_clean['RENDIMIENTO_GLOBAL']

#Dividir dataset para entrenar y validar el modelo
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

cat_features = X_train.select_dtypes(include=['object']).columns.tolist()

#entrenar con catboost
from catboost import CatBoostClassifier

model = CatBoostClassifier(
    iterations=1500,
    learning_rate=0.03,
    depth=6,
    loss_function='MultiClass',
    cat_features=cat_features,
    random_seed=42,
    verbose=200,
    early_stopping_rounds=100
)
model.fit(
    X_train, y_train,
    eval_set=(X_val, y_val),
    cat_features=cat_features
)

0:	learn: 1.3642285	test: 1.3645411	best: 1.3645411 (0)	total: 2.26s	remaining: 18m 47s
100:	learn: 1.2008224	test: 1.2022323	best: 1.2022323 (100)	total: 2m 53s	remaining: 11m 25s
200:	learn: 1.1923122	test: 1.1955217	best: 1.1955217 (200)	total: 5m 44s	remaining: 8m 31s
300:	learn: 1.1882407	test: 1.1932920	best: 1.1932920 (300)	total: 8m 30s	remaining: 5m 37s
400:	learn: 1.1853196	test: 1.1921736	best: 1.1921736 (400)	total: 11m 20s	remaining: 2m 48s
499:	learn: 1.1830883	test: 1.1917865	best: 1.1917865 (499)	total: 14m 13s	remaining: 0us

bestTest = 1.191786504
bestIteration = 499



In [ ]:
model.fit(
    X, y,
    cat_features=cat_features
)

0:	learn: 1.3632737	total: 2.3s	remaining: 19m 7s
100:	learn: 1.2009446	total: 3m 20s	remaining: 13m 12s
200:	learn: 1.1925583	total: 6m 45s	remaining: 10m 2s
300:	learn: 1.1885829	total: 10m 3s	remaining: 6m 38s
400:	learn: 1.1860938	total: 13m 24s	remaining: 3m 18s
499:	learn: 1.1839925	total: 16m 44s	remaining: 0us


**Carga y preprocesado de los datos del archivo test**

In [ ]:
test = pd.read_csv("test.csv")
test_ids = test["ID"]  # Guardamos los IDs como referencia
test_clean = test.copy()

test_clean = test_clean.drop(columns=['ID', 'PERIODO_ACADEMICO'])

# Separar valores numéricos y no numéricos
numeric_cols = test_clean.select_dtypes(include=np.number).columns
non_numeric_cols = test_clean.select_dtypes(exclude=np.number).columns

# Reemplazar valores nulos de columnas numéricas con la media
test_clean[numeric_cols] = test_clean[numeric_cols].fillna(test_clean[numeric_cols].median())

# Reemplazar valores nulos de columnas no numéricas por la moda
for col in non_numeric_cols:
    test_clean[col] = test_clean[col].fillna(test_clean[col].mode()[0])

test_clean.isna().sum()

# Reemplazar valores si & no por 0 & 1
def replace_binary(col):
    return col.replace({
        'Si': 1, 'Sí': 1, 'S': 1, 'Y': 1, '1': 1, 'si': 1, 'sí': 1,
        'No': 0, 'N': 0, '0': 0, 'no': 0
    })

binary_cols = ['F_TIENEINTERNET', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL',
               'E_PRIVADO_LIBERTAD', 'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR',
               'F_TIENEINTERNET.1']

for col in binary_cols:
    test_clean[col] = replace_binary(test_clean[col])

test_clean[binary_cols] = test_clean[binary_cols].astype(int)

# Reemplazar valores de variables ordinales
test_clean['F_EDUCACIONPADRE'] = test_clean['F_EDUCACIONPADRE'].replace(['No sabe', 'No Aplica'], 'Desconocido')
test_clean['F_EDUCACIONMADRE'] = test_clean['F_EDUCACIONMADRE'].replace(['No sabe', 'No Aplica'], 'Desconocido')

orden_educacion = [
    'Desconocido',
    'Ninguno',
    'Primaria incompleta',
    'Primaria completa',
    'Secundaria (Bachillerato) incompleta',
    'Secundaria (Bachillerato) completa',
    'Técnica o tecnológica incompleta',
    'Técnica o tecnológica completa',
    'Educación profesional incompleta',
    'Educación profesional completa',
    'Postgrado'
]
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder(categories=[orden_educacion])
test_clean['F_EDUCACIONPADRE'] = encoder.fit_transform(test_clean[['F_EDUCACIONPADRE']])
test_clean['F_EDUCACIONMADRE'] = encoder.fit_transform(test_clean[['F_EDUCACIONMADRE']])

orden_matrícula = [
    'No pagó matrícula',
    'Menos de 500 mil',
    'Entre 500 mil y menos de 1 millón',
    'Entre 1 millón y menos de 2.5 millones',
    'Entre 2.5 millones y menos de 4 millones',
    'Entre 4 millones y menos de 5.5 millones',
    'Entre 5.5 millones y menos de 7 millones',
    'Más de 7 millones'
]
encoder = OrdinalEncoder(categories=[orden_matrícula])
test_clean['E_VALORMATRICULAUNIVERSIDAD'] = encoder.fit_transform(test_clean[['E_VALORMATRICULAUNIVERSIDAD']])

orden_estrato = [
    'Sin Estrato',
    'Estrato 1',
    'Estrato 2',
    'Estrato 3',
    'Estrato 4',
    'Estrato 5',
    'Estrato 6'
]
encoder = OrdinalEncoder(categories=[orden_estrato])
test_clean['F_ESTRATOVIVIENDA'] = encoder.fit_transform(test_clean[['F_ESTRATOVIVIENDA']])

orden_trabajo = [
    '0',
    'Menos de 10 horas',
    'Entre 11 y 20 horas',
    'Entre 21 y 30 horas',
    'Más de 30 horas'
]
encoder = OrdinalEncoder(categories=[orden_trabajo])
test_clean['E_HORASSEMANATRABAJA'] = encoder.fit_transform(test_clean[['E_HORASSEMANATRABAJA']])

# Con media = 0, desviación estándar = 1
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
cols_ordinales = ['E_HORASSEMANATRABAJA', 'F_ESTRATOVIVIENDA',
                  'E_VALORMATRICULAUNIVERSIDAD', 'F_EDUCACIONPADRE', 'F_EDUCACIONMADRE']

test_clean[cols_ordinales] = scaler.fit_transform(test_clean[cols_ordinales])

# Variables numéricas
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
cols_numericas = ['INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']

test_clean[cols_numericas] = scaler.fit_transform(test_clean[cols_numericas])

/tmp/ipython-input-3046738009.py:22: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return col.replace({


**Predicciones**

In [ ]:
test_predictions = model.predict(test_clean)

**Preparando el archivo solución**

In [ ]:
map_dict = {0: 'bajo', 1: 'medio-bajo', 2: 'medio-alto', 3: 'alto'}
test_predictions_cat = [map_dict[i[0]] for i in test_predictions]

solucion = pd.DataFrame({
    'ID': test_ids,
    'RENDIMIENTO_GLOBAL': test_predictions_cat
})
solucion.to_csv('submission1.csv', index=False)